In [ ]:
!pip install python-binance
import pandas as pd
import sqlalchemy
from binance.client import Client
from binance import BinanceSocketManager

In [15]:
api_key = "9ZC22lmAGrIh51mHVUjnLnNvWXE60zsKuI8YqitHg3ucTzTUpq6bolz8iIESNAlL"
api_secret = "C18C1xRmBaXmjE7hbEC703gcnRDvSkK27AjBivt6PJXoVEoxv6QaH8DXPwcXGdLv"

In [16]:
client = Client(api_key,api_secret)

In [ ]:
client.get_account()

In [ ]:
client.get_historical_klines('BTCUSDT', '1m', '30 m ago UTC')

In [ ]:
pd.DataFrame(client.get_historical_klines('BTCUSDT', '1m', '30 min ago UTC'))

In [27]:
def getminutedata(symbol, interval, lookback):
  frame = pd.DataFrame(client.get_historical_klines(symbol, interval, lookback+' min ago UTC'))
  frame = frame.iloc[:,:6]
  frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
  frame = frame.set_index('Time')
  frame.index = pd.to_datetime(frame.index, unit = 'ms')
  frame = frame.astype(float)
  return frame

In [28]:
test=getminutedata('BTCUSDT', '1m', '30')

In [ ]:
test.Open.plot()

In [ ]:
#buy if asset fell by more then 0.2% within the last 30 min
#sell if asset rise by more then 0.15% or falls further by 0.15%

In [43]:
def strategytest(symbol, qty, entried=False):
  df = getminutedata(symbol, '1m', '30m')
  cumulret = (df.Open.pct_change()+1).cumprod() - 1
  if not entried:
    if cumulret[-1] < -0.001:
      order = client.create_order(symbol=symbol, side='BUY', type='MARKET', quantity=qty)
      print(order)
      entried = True
    else:
      print('No Trade has been executed')
  if entried:
    while True:
      df = getminutedata(symbol, '1m', '30m')
      sincebuy = df.loc[df.index > pd.to_datetime(order['transactTime'], unit='ms')]  
      if len(sincebuy)>0:
        sincebuyret =  (sincebuy.Open.pct_change()+1).cumprod() - 1
        if sincebuyret[-1] > 0.0015 or sincebuyret[-1]< -0.0015:
          order = client.create_order(symbol=symbol, side='SELL', type='MARKET', quantity=qty)  
          print(order)
          break



In [ ]:
strategytest('BTCUSDT', 0.002)